In [77]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [78]:
districts1 = []
streets1 = []
room_num1 = []
flat_areas1 = []
is_studio1 = []
prices1 = []
fees1 = []

In [79]:
def append_to_outterlist(innerlist, outterlist):
    for el in innerlist:
        outterlist.append(el)

In [80]:
url = 'https://www.bezrealitky.cz/vyhledat?offerType=PRONAJEM&estateType=BYT&osm_value=Hlavn%C3%AD+m%C4%9Bsto+Praha%2C+Praha%2C+%C4%8Cesko&regionOsmIds=R435514&currency=CZK&page='

In [89]:
#-----------------scraping 3 pages-----------------
for page in range(1, 4):
    webpage = requests.get(url + str(page))
    soup = BeautifulSoup(webpage.content, 'html.parser')
    
    #-----------------removes not necessary "Výhodná nabídka" box-----------------
    section_tags = soup.find_all('section', class_="box Section_section__gjwvr section PropertyCardCarousel_propertyCardCarouselSection__ts7NT mb-0 bg-grey-ultra-light Box_box--rounded-lg__uS5ZG")
    for section in section_tags:
        section.decompose()

In [90]:
    #-----------------Prague districts and streets-----------------
    streets_districts = [district.get_text() for district in soup.find_all('span', class_="PropertyCard_propertyCardAddress__hNqyR text-subheadline text-truncate")]
    streets = [street.split(',')[0] for street in streets_districts]
    districts = [district.split(',')[-1].split('- ')[-1] for district in streets_districts]
    
    
    #-----------------number of rooms, areas of flat-----------------
    flat_info = [area.get_text() for area in soup.find_all('li', class_="FeaturesList_featuresListItem__RYf_f")]
    flat_info = list(map(lambda x: x.replace('Garsoniéra', '1+kk'), flat_info))

    flat_areas = list(map(lambda x: int(x.split('\xa0')[0]), flat_info[1::2]))

    
    room_num = list(map(lambda x: int(x.split('+')[0]), flat_info[0::2]))

    
    is_studio = ['Yes' if x.split('+')[1] == 'kk' else 'No' for x in flat_info[0::2]]

    
    #-----------------prices and fees-----------------
    price_infos = soup.find_all('div', class_="PropertyPrice_propertyPrice__lthza propertyPrice mb-0 mt-3")
    prices = []
    fees = []
    for tag in price_infos:
    
        rent_price = tag.find('span', class_="PropertyPrice_propertyPriceAmount__WdEE1")
        prices.append(rent_price)
    
        add_fee = tag.find('span', class_="PropertyPrice_propertyPriceAdditional__5jYQ6")
        fees.append(add_fee)
        
    prices = list(map(lambda x: int(x.get_text().replace('\xa0', '').split('Kč')[0]), prices))
     
    
    fees = list(map(lambda x: ' + 0\xa0Kč' if x is None else x.get_text(), fees))
    fees = list(map(lambda x : int(x.replace('\xa0', '').split('Kč')[0].strip(' +')), fees))


In [93]:
    append_to_outterlist(streets, streets1)
    append_to_outterlist(districts, districts1)
    append_to_outterlist(flat_areas, flat_areas1)
    append_to_outterlist(room_num, room_num1)
    append_to_outterlist(is_studio, is_studio1)
    append_to_outterlist(prices, prices1)
    append_to_outterlist(fees, fees1)

In [94]:
flat_dict = {'districts': districts1,
             'streets': streets1,
             'room_num': room_num1,
             'area': flat_areas1,
             'is_studio': is_studio1, 
             'prices': prices1,
             'fees': fees1}

{'districts': ['Háje', 'Michle', 'Hlubočepy', 'Vysočany', 'Vinohrady', 'Vršovice', 'Žižkov', 'Michle', 'Košíře', 'Hostivař', 'Michle', 'Žižkov', 'Nové Město', 'Smíchov', 'Smíchov', 'Háje', 'Michle', 'Hlubočepy', 'Vysočany', 'Vinohrady', 'Vršovice', 'Žižkov', 'Michle', 'Košíře', 'Hostivař', 'Michle', 'Žižkov', 'Nové Město', 'Smíchov', 'Smíchov'], 'streets': ['Anny Drabíkové', 'Pod Dálnicí', 'Voskovcova', 'Freyova', 'Mánesova', 'Sevastopolská', 'Čajkovského', 'Bítovská', 'Naskové', 'Záveská', 'V\xa0Dolině', 'Na Vrcholu', 'Lublaňská', 'Nad Koulkou', 'Na Plzeňce', 'Anny Drabíkové', 'Pod Dálnicí', 'Voskovcova', 'Freyova', 'Mánesova', 'Sevastopolská', 'Čajkovského', 'Bítovská', 'Naskové', 'Záveská', 'V\xa0Dolině', 'Na Vrcholu', 'Lublaňská', 'Nad Koulkou', 'Na Plzeňce'], 'room_num': [3, 3, 3, 1, 3, 3, 1, 2, 2, 3, 3, 2, 1, 2, 1, 3, 3, 3, 1, 3, 3, 1, 2, 2, 3, 3, 2, 1, 2, 1], 'area': [60, 85, 88, 24, 105, 105, 58, 41, 59, 60, 92, 52, 35, 50, 39, 60, 85, 88, 24, 105, 105, 58, 41, 59, 60, 92, 52, 

In [96]:
bezrealitka_df = pd.DataFrame.from_dict(flat_dict)
print(bezrealitka_df)

     districts         streets  room_num  area is_studio  prices  fees
0         Háje  Anny Drabíkové         3    60       Yes   23990  1879
1       Michle     Pod Dálnicí         3    85        No   26000  5000
2    Hlubočepy      Voskovcova         3    88       Yes   29000  7920
3     Vysočany         Freyova         1    24       Yes   12500  3405
4    Vinohrady        Mánesova         3   105        No   44500  9500
5     Vršovice   Sevastopolská         3   105       Yes   36000     0
6       Žižkov     Čajkovského         1    58        No   22500  4000
7       Michle        Bítovská         2    41       Yes   21000  1500
8       Košíře         Naskové         2    59       Yes   25000  5000
9     Hostivař         Záveská         3    60       Yes   20900  6500
10      Michle        V Dolině         3    92        No    9000     0
11      Žižkov      Na Vrcholu         2    52        No   21500  5800
12  Nové Město       Lublaňská         1    35       Yes   14000  4000
13    